# Transmission line simulation with pysonnet
Let's do some imports

In [1]:
import logging
import numpy as np
import pysonnet as ps
# gdspy is availible via 'pip install gdspy'
# or at https://github.com/heitzmann/gdspy
import gdspy as gp

## Make a simple geometry with gdspy
We can make a cool wavy transmission line using the gdspy.Path class

The result can be easily viewed using gdspy's built in layout viewer (uncomment the last line)

In [2]:
# geometric constants
box_x, box_y = 100, 100  # size of the box
width = 5  # transmission line width
layer = 0  # layer for gds format
datatype = 1  # datatype for gds format
# initialize the path
path = gp.Path(width, initial_point=(box_x / 3, 0))
# define the cosine path and it's derivative
def path_function(t):
    return (-box_x / 6 * (np.cos(np.pi * t) - 1), t * box_y)
def d_path_function(t):
    return (np.pi * box_x / 6 * np.sin(np.pi * t), box_y)
path.parametric(path_function, curve_derivative=d_path_function,
                layer=layer, datatype=datatype)
# add the path to a cell
cell = gp.Cell('feedline', exclude_from_current=True)
cell.add(path);
# show the result
# gp.LayoutViewer(cells=cell)

## Configure the project settings
The first step is to pick the box settings and the materials that we will use for the sonnet simulation

In [3]:
# The GeometryProject is the most basic Sonnet project type
project = ps.GeometryProject()

# I don't like the default Sonnet units, so let's change them
project.set_units(length='um')  # the default length unit for Sonnet is mils

# Then we can set up the box
project.setup_box(box_x, box_y, 500, 500)

# Next is defining the metal types and if they are on the box top/bottom
project.define_metal("general", "PtSi", ls=21)  # high kinetic inductance superconductor
project.define_metal("general", "Nb", ls=0.08)  # low kinetic inductance superconductor
project.set_box_cover("free space", top=True)
project.set_box_cover("custom", name="Nb", bottom=True)

# Lets add a dielectric layer under the microstrip and air above
project.add_dielectric("air", layer , thickness=1000)
project.add_dielectric("silicon", layer + 1, thickness=100, epsilon=11.9, dielectric_loss=0.004, conductivity=4.4e-4)

# We can also define technology layers
project.define_technology_layer("metal", "microstrip", layer, "PtSi", fill_type="diagonal")

## Load the geometry into a pysonnet project
The polygons can be extracted using the get_polygons() method for the Cell, CellReference or CellArray.

The by_spec keyword is useful for separating different layers and datatypes to assign groups of polygons to different Sonnet layers with different properties. 

In [4]:
polygons = cell.get_polygons(by_spec=True)
microstrip_polygons = polygons[(layer, datatype)]  # pick the polygons that I want

We can now add the polygons and ports to our project.

In [5]:
project.add_polygons("metal", microstrip_polygons, tech_layer="microstrip")
project.add_port("standard", 1, 2 * box_x / 3, box_y, resistance=50)
project.add_port("standard", 2, box_x / 3, 0, resistance=50)

## Set up the simulation
After defining the box, materials, geometry, and ports, we can choose what kind of simulation we are doing.